# Reconocimiento de fonemas usando CTC

In [7]:
import json
import torch
import torch.utils.data as data
import torchaudio
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import  Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Implementación del dataset y el dataloader

### Dataset
Me convierte los datos crudos para que puedan ser usados por `Dataloader`. Me permite implementar una función `__getitem__()` en la cual leemos los datos y devolvemos por ejemplo el wav y la transcripción de cada dato.

In [ ]:
train_json = 'data/train.json'
test_json = 'data/test.json'
valid_json = 'data/dev.json'

class TimitDataset(Dataset):
    def __init__(self, json_file):
        with open(json_file, 'r') as f:
            self.datos_json = json.load(f)

    def __len__(self):
        return len(self.datos_json)
    
    def __getitem__(self,key):
        wavdir = self.datos_json[key]['wav']
        duration = self.datos_json[key]['duration']
        phn = self.datos_json[key]['phn']
        return wavdir, duration, phn

def collate_fn(batch):
    # El batch es una lista de tuplas: [(dato1,label1), (dato2,label2),...]
    sequences, labels = zip(*batch) # Esto devuelve: 
                                    # sequences = (dato1,dato2,...)
                                    # labels = (label1,label2,...)
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0)
    labels = torch.tensor(labels)
    return padded_sequences, labels # Esta es la salida del dataloader
   
train_ds = TimitDataset(train_json)
test_ds = TimitDataset(test_json)
valid_ds = TimitDataset(valid_json)

train_dl = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate_fn)
test_dl = DataLoader(train_ds, batch_size=16, shuffle=False, collate_fn=collate_fn)
valid_dl = DataLoader(train_ds, batch_size=16, shuffle=False, collate_fn=collate_fn)

for batch in train_dl:
    print(batch[1])  # Imprime las etiquetas del batch
        

In [ ]:
train_ds[0]

In [15]:
len(dataloader)

462

In [ ]:
wavdir, dur, phn = valid_dataset['mrws1_sx230']
phn

'sil ah l aw l iy w ey hh iy er sil b ah r ae sh sil n l ay z aa l eh r er z sil'

### DataLoader
La función principal es agrupar en batches los items que provienen del dataset y, si es necesario, padear los items para que el batch sea de igual ancho.